In [1]:
import graphlab as gl

A newer version of GraphLab Create (v1.9) is available! Your current version is v1.8.5.

You can use pip to upgrade the graphlab-create package. For more information see https://dato.com/products/create/upgrade.


In [2]:
import math
from collections import OrderedDict

In [3]:
stack1 = gl.SFrame('MyData1.csv')

2016-05-03 21:42:47,166 [INFO] graphlab.cython.cy_server, 176: GraphLab Create v1.8.5 started. Logging: /tmp/graphlab_server_1462336961.log


Unable to parse line "18671,"PHP replace double backslashes ""\\"" to a single backslash ""\""",php"

Unable to parse line "3747,"What is the difference between the Project and SVN workingDirectory config Blocks in CruiseControl.NET"

Unable to parse line "",svn"

Unable to parse line "16398,"Using ""~"" in a path resolvs as C:\",asp.net"

...

Unable to parse line "",ruby"

46 lines failed to parse correctly

Finished parsing file /Users/saur_navigator/Desktop/csv/MyData1.csv

Parsing completed. Parsed 558919 lines in 1.46704 secs.


Inferred types from first line of file as 
column_type_hints=[int,str,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


In [4]:
N = 200
stack = stack1[1:N+1]

In [17]:
# corpus row for each element 
corpus_row = []
tf_row = []
df_row = {}
s = set()
# total number of unique words
uniq_word = []
tf_idf = []
def calUniqueWord(text):
    for i in range(0,N):
        corpus_row.append(text[i]['Title'].split(' '))
        for j in corpus_row[i]:
            s.add(j)
    uniq_word = list(s)
    uniq_word.sort()
    return uniq_word

# to calculate term frequesncy for each document 
def getTf(corpus_row):
    for i in range (0,N):
        list_row = corpus_row[i]
        dic_row = {}
        for i in list_row :
            if i in dic_row.keys():
                dic_row[i] += 1
            else:
                dic_row[i]=1
        tf_row.append(dic_row)

# to calculate df in number of dcoument
def getDf(corpus_row):
    for i in range(0,N):
        for j in uniq_word:
            if j in corpus_row[i]:
                if j in df_row.keys():
                    df_row[j] += 1
                else:
                    df_row[j]=1

def getTfIdf(corpus_row):
    for i in tf_row:
        dic = i;
        temp = {}
        for key in dic.keys():
            temp[key] = dic[key]*(math.log(N/df_row[key]))
        tf_idf.append(temp)
    #print tf_idf
    
def getDistance(a,b):
    distance = 0;
    setA = set(a.keys())
    setB = set(b.keys())
    setAnB = setA & setB
    for x in setAnB:
        distance = distance + pow(a[x]-b[x],2)
    distance = math.sqrt(distance)
    return distance


def setNeighbour(testSet,trainRow,k):
    distance = [];
    cal_dis = 0
    for x in range(0,len(trainRow)):
        dic = {}
        cal_dist = getDistance(testSet['tf'],trainRow[x]['tf'])
        dic = {cal_dist:trainRow[x]['Tag1']}
        distance.append(dic)
    distance = sorted(distance,reverse=True,key=(lambda x: x.keys()[0]))
    k_neighbour = []
    for i in range(0,k):
        k_neighbour.append(distance[i].values()[0])
    return k_neighbour
    
def getPrediction(k_neighbour):
    dic = {};
    for x in k_neighbour:
        if x in dic.keys():
            dic[x] = dic[x]+1
        else:
            dic[x] = 1
    dic = OrderedDict(sorted(dic.items(),key=lambda x: x[0],reverse=True))
    return dic.keys()[0]

def getAccuracy(testSet,predict):
    accuracy = 0;
    for x in range(0,len(testSet)):
        if testSet[x] == predict[x]:
            accuracy = accuracy + 1
    if accuracy == 0:
        return 0
    return accuracy/float(len(testSet))

In [18]:
uniq_word = calUniqueWord(stack)
getTf(corpus_row)

In [19]:
getDf(corpus_row)

In [20]:
getTfIdf(corpus_row)
print len(tf_idf)

200


In [21]:
def incTag(arr):
    tag = arr['Tag1']
    dic = arr['tf']
    val = sorted(dic.values())
    val = val[len(dic)-1]
    for key in dic.keys() :
        if key == tag:
            dic[key] = val
    return arr['tf']

def removeVagueWord(arr):
    dic = arr['word_count']
    list_vag = ['a','the','and','in','for','of','to','how','i','from','why','is','use','do','you'];
    for key in list_vag:
        if key in dic.keys():
            del dic[key]
    return arr['word_count']

In [141]:
# stack['word_count'] = gl.text_analytics.count_words(stack['Title'])
#stack['word_count'] = stack.apply(removeVagueWord)

In [23]:
stack['tf'] = tf_idf
stack['tf'] = stack.apply(incTag)

In [24]:
train , test = stack.random_split(.9,.1)
train.head()

OwnerUserId,Title,Tag1,tf
9,Percentage width child inabsolutely positioned ...,html,"{""doesn't"":4.605170185988092, 'in': ..."
9,Tools for porting J# codeto C# ...,j#,"{'code':3.912023005428146, 'f ..."
1,How do I calculatesomeone's age in c#? ...,c#,"{'do': 1.791759469228055,'c#?': 4.605170185988 ..."
1021610,retrieve data fromNSUserDefaults to ...,iphone,"{'retrieve':5.298317366548036, ..."
33,Reliable Timer in aConsole Application ...,c#,"{'a': 1.3862943611198906,'Console': ..."
48,Latest information on PHPupcoming releases ...,php,"{'information':5.298317366548036, ..."
22,Throw Error In MySQLTrigger ...,mysql,"{'Error':5.298317366548036, ..."
23,How to use the C socketAPI in C++ ...,c++,"{'C': 5.298317366548036,'socket': ..."
1,How do I calculaterelative time? ...,c#,"{'do': 1.791759469228055,'time?': ..."
9,Determining web user'stime zone ...,html,"{'Determining':5.298317366548036, ..."


In [25]:
train_nw = gl.SFrame({'Tag1': train['Tag1'],
                      'tf': train['tf']})
test_nw = gl.SFrame({'Tag1': test['Tag1'],
                      'tf': test['tf']})

In [26]:
# result will store the label predicted by the KNN model
result = []
for x in range(0,len(test_nw)):
    result.append(getPrediction(setNeighbour(test_nw[x],train_nw,10)))

In [27]:
# KNN evaludating model to calculate accuracy
getAccuracy(result,test_nw['Tag1'])

0.11764705882352941

In [21]:
## graphlab function to create KNN classifier
kmm_model = gl.nearest_neighbor_classifier.create(train_nw,target='Tag1',features=['tf'])

Starting brute force nearest neighbors model training.

In [103]:
## graphlab function to evaluate KNN classifier
ans = kmm_model.evaluate(test_nw,max_neighbors=20,metric='accuracy')
print ans

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 17      | 0.546448    | 930us        |

...

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 17      | 0.546448    | 504us        |

| Done         |         | 100         | 1.043ms      |

+--------------+---------+-------------+--------------+